In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np; np.set_printoptions(precision=2, threshold=100); np.random.seed(0)
import torch; torch.set_printoptions(precision=2, threshold=100)
seed = 1 

import matplotlib.pyplot as plt; plt.rc('font', size=12)
import matplotlib 
from matplotlib.font_manager import FontProperties
from mpl_toolkits import mplot3d
import matplotlib.pylab as plt

import time
import itertools
import random; random.seed(0)
import datetime
import pickle
import copy
import scipy
import os
import sys

from task import *
from functions import *
from train import *
from model import *

print(torch.__version__)
print(sys.version)
                
%matplotlib inline

# Run the model on a sequence of trials and save the results

In [ ]:
import pickle

start = time.time()

plt.rc('font', size=12)

filename = '_testdata_noiseless'

for model_name in sorted(os.listdir('/where/models/are/stored/')):
    if ('2023-05-10' in model_name) and 'success' in model_name:    # you can subselect models here
        print('\n'+model_name)
    
        if model_name+filename in os.listdir('/where/data/for/test/run/will/be/stored/'):    # where the data for the test run will be stored
            print('already saved test data, pass')
            continue
            
        path_to_file = '/where/models/are/stored/' + model_name    # where model is stored
        path_to_pickle_file = '/where/data/for/test/run/will/be/stored/' + model_name + filename    # where the data for the test run will be stored
        
        with HiddenPrints():
            model, hp_test, hp_task_test, optim, saved_data = load_model_v2(path_to_file=path_to_file, model_name=model_name, simple=False, plot=False, toprint=False)
        
        # make noiseless
        model.rnn.network_noise = 0
        hp_test['input_noise_perceptual'] = 0
        hp_test['input_noise_rule'] = 0
         
        test_data = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=1, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
                                              compute_current=False, random_switch=False, n_switches=10, concat_activity=False)
        with open(path_to_pickle_file, 'wb') as f:  
            pickle.dump(test_data, f)
            
print(time.time()-start)

# Optogenetic silencing of SST neurons in the sensorimotor module

In [ ]:
import pickle

start = time.time()

plt.rc('font', size=12)
    
for model_name in sorted(os.listdir('/scratch/yl4317/two_module_rnn/saved_models/')):
    if ('2023-05-10' in model_name) and ('success' in model_name): 
        print('\n'+model_name)
        if model_name+'_testdata_silenceSRSST_noiseless' in os.listdir('/where/data/for/test/run/will/be/stored/'):
            print('already saved test data, pass')
            continue
        path_to_file = '/where/models/are/stored/' + model_name
        path_to_pickle_file = '/where/data/for/test/run/will/be/stored/' + model_name + '_testdata_silenceSRSST_noiseless'
        with HiddenPrints():
            model, hp_test, hp_task_test, optim, saved_data = load_model_v2(path_to_file=path_to_file, model_name=model_name, simple=False, plot=False, toprint=False)
        
        
        # make noiseless
        model.rnn.network_noise = 0
        hp_test['input_noise_perceptual'] = 0
        hp_test['input_noise_rule'] = 0
        
        # opto params
        opto_params_sst = {'t': np.arange(hp_task_test['trial_start']//hp_test['dt'], hp_task_test['trial_end']//hp_test['dt']), 
                           'neuron_idx': model.rnn.cg_idx['sr_sst'],
                           'value': torch.tensor(0).to(torch.float32)}
        
        # change ITI
        iti = 1000
        hp_task_change_iti = copy.deepcopy(hp_task_test)
        for key in ['center_card_on', 'center_card_off', 'test_cards_on', 'test_cards_off', 'resp_start', 'resp_end', 'trial_end']:
            hp_task_change_iti[key] += iti - (hp_task_test['center_card_on'] - hp_task_test['trial_history_end'])    # adjust the ITI 
        hp_task_test = hp_task_change_iti
        
        
        test_data_silencing_sst = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=1, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
                                                              compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_sst)
        with open(path_to_pickle_file, 'wb') as f:
            pickle.dump(test_data_silencing_sst, f)
            
print(time.time()-start)

# Optogenetic silencing of VIP neurons in the sensorimotor module

In [ ]:
import pickle

start = time.time()

plt.rc('font', size=12)
    
for model_name in sorted(os.listdir('/scratch/yl4317/two_module_rnn/saved_models/')):
    if ('2023-05-10' in model_name) and ('success' in model_name): 
        print('\n'+model_name)
        if model_name+'_testdata_silenceSRVIP_noiseless' in os.listdir('/where/data/for/test/run/will/be/stored/'):
            print('already saved test data, pass')
            continue
        path_to_file = '/where/models/are/stored/' + model_name
        path_to_pickle_file = '/where/data/for/test/run/will/be/stored/' + model_name + '_testdata_silenceSRVIP_noiseless'
        with HiddenPrints():
            model, hp_test, hp_task_test, optim, saved_data = load_model_v2(path_to_file=path_to_file, model_name=model_name, simple=False, plot=False, toprint=False)
        
        
        # make noiseless
        model.rnn.network_noise = 0
        hp_test['input_noise_perceptual'] = 0
        hp_test['input_noise_rule'] = 0
        
        # opto params
        opto_params_vip = {'t': np.arange(hp_task_test['trial_start']//hp_test['dt'], hp_task_test['trial_end']//hp_test['dt']), 
                           'neuron_idx': model.rnn.cg_idx['sr_vip'],
                           'value': torch.tensor(0).to(torch.float32)} 
        
        # change ITI
        iti = 1000
        hp_task_change_iti = copy.deepcopy(hp_task_test)
        for key in ['center_card_on', 'center_card_off', 'test_cards_on', 'test_cards_off', 'resp_start', 'resp_end', 'trial_end']:
            hp_task_change_iti[key] += iti - (hp_task_test['center_card_on'] - hp_task_test['trial_history_end'])    # adjust the ITI 
        hp_task_test = hp_task_change_iti
        
        test_data_silencing_vip = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=1, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
                                                              compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_vip)
        
        with open(path_to_pickle_file, 'wb') as f:
            pickle.dump(test_data_silencing_vip, f)
            
print(time.time()-start)

# Optogenetic silencing of PV neurons in the sensorimotor module

In [ ]:
import pickle

start = time.time()

plt.rc('font', size=12)
    
for model_name in sorted(os.listdir('/scratch/yl4317/two_module_rnn/saved_models/')):
    if ('2023-05-10' in model_name) and ('success' in model_name): 
        print('\n'+model_name)
        if model_name+'_testdata_silenceSRPV_noiseless' in os.listdir('/where/data/for/test/run/will/be/stored/'):
            print('already saved test data, pass')
            continue
        path_to_file = '/where/models/are/stored/' + model_name
        path_to_pickle_file = '/where/data/for/test/run/will/be/stored/' + model_name + '_testdata_silenceSRPV_noiseless'
        with HiddenPrints():
            model, hp_test, hp_task_test, optim, saved_data = load_model_v2(path_to_file=path_to_file, model_name=model_name, simple=False, plot=False, toprint=False)
        
        
        # make noiseless
        model.rnn.network_noise = 0
        hp_test['input_noise_perceptual'] = 0
        hp_test['input_noise_rule'] = 0
        
        # opto params
        opto_params_vip = {'t': np.arange(hp_task_test['trial_start']//hp_test['dt'], hp_task_test['trial_end']//hp_test['dt']), 
                           'neuron_idx': model.rnn.cg_idx['sr_pv'],
                           'value': torch.tensor(0).to(torch.float32)} 
        
        # change ITI
        iti = 1000
        hp_task_change_iti = copy.deepcopy(hp_task_test)
        for key in ['center_card_on', 'center_card_off', 'test_cards_on', 'test_cards_off', 'resp_start', 'resp_end', 'trial_end']:
            hp_task_change_iti[key] += iti - (hp_task_test['center_card_on'] - hp_task_test['trial_history_end'])    # adjust the ITI 
        hp_task_test = hp_task_change_iti
        
        test_data_silencing_pv = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=10, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
                                                              compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_vip)
        
        with open(path_to_pickle_file, 'wb') as f:
            pickle.dump(test_data_silencing_pv, f)
            
print(time.time()-start)